<a href="https://www.kaggle.com/code/vtrackstar/nba-player-stats-through-12-6-data-science-project?scriptVersionId=212229313" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Ask
* Determine the best statistical mid-range shooters (10 feet up to the three-point line) and defenders in terms of their position group as Guards, Wings and Bigs.

## Prepare

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nbaplayerstatsthru126/2024 NBA Scoring (12_6) - Shooting.csv
/kaggle/input/nbaplayerstatsthru126/2024 NBA Scoring (12_6) - PerGame.csv
/kaggle/input/nbaplayerstatsthru126/2024 NBA Scoring (12_6) - Play by Play.csv
/kaggle/input/nbaplayerstatsthru126/2024 NBA Scoring (12_6) - Advanced.csv


In [2]:
perg = pd.read_csv('/kaggle/input/nbaplayerstatsthru126/2024 NBA Scoring (12_6) - PerGame.csv')
pbp = pd.read_csv('/kaggle/input/nbaplayerstatsthru126/2024 NBA Scoring (12_6) - Play by Play.csv')
shoot = pd.read_csv('/kaggle/input/nbaplayerstatsthru126/2024 NBA Scoring (12_6) - Shooting.csv')
adv = pd.read_csv('/kaggle/input/nbaplayerstatsthru126/2024 NBA Scoring (12_6) - Advanced.csv')

In [3]:
print(perg.head())
perg.columns

   Rk                   Player  Age Team Pos   G  GS    MP    FG   FGA  ...  \
0   1    Giannis Antetokounmpo   30  MIL  PF  20  20  34.9  12.8  20.9  ...   
1   2              LaMelo Ball   23  CHO  PG  18  18  34.1  10.7  24.9  ...   
2   3             Nikola Jokić   29  DEN   C  17  17  37.7  11.4  20.4  ...   
3   4  Shai Gilgeous-Alexander   26  OKC  PG  22  22  34.1  10.5  20.6  ...   
4   5           Paolo Banchero   22  ORL  PF   5   5  36.4   9.6  19.4  ...   

   ORB  DRB   TRB   AST  STL  BLK  TOV   PF   PTS  Awards  
0  2.1  9.5  11.6   6.4  0.5  1.3  3.4  2.5  32.5     NaN  
1  0.9  4.4   5.4   6.9  1.1  0.2  4.5  4.1  31.1     NaN  
2  3.9  9.5  13.4  10.4  1.8  0.8  3.7  1.9  29.9     NaN  
3  0.9  4.5   5.4   6.5  1.8  1.0  2.6  2.0  29.8     NaN  
4  2.4  6.4   8.8   5.6  0.6  0.8  2.2  2.6  29.0     NaN  

[5 rows x 31 columns]


Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Awards'],
      dtype='object')

In [4]:
print(pbp.head())
pbp.columns

   Rk             Player  Age Team Pos   G  GS   MP   PG%   SG%  ...  \
0   1       De'Aaron Fox   27  SAC  PG  24  24  900  93.0   7.0  ...   
1   2      Keegan Murray   24  SAC  PF  24  24  885   0.0   0.0  ...   
2   3  Tyrese Haliburton   24  IND  PG  24  24  847  58.0  41.0  ...   
3   4      Mikal Bridges   28  NYK  SF  22  22  841   0.0   0.0  ...   
4   5      Jalen Johnson   23  ATL  SF  23  23  836   0.0   0.0  ...   

   BadPassTO  LostBallTO  ShootFoulCmtd  OffFoulCmtd  ShootFoulDrawn  \
0         36          38             16            6              72   
1         11          11             37            2              16   
2         32          12             20            1              29   
3         20          11             13            2              10   
4         42          12             24           10              35   

   OffFoulDrawn  PtsGenbyAsts  And1s  FGA_Blkd  Awards  
0            15           345     11        25     NaN  
1             8     

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PG%', 'SG%',
       'SF%', 'PF%', 'C%', 'OnCourt', 'On-Off', 'BadPassTO', 'LostBallTO',
       'ShootFoulCmtd', 'OffFoulCmtd', 'ShootFoulDrawn', 'OffFoulDrawn',
       'PtsGenbyAsts', 'And1s', 'FGA_Blkd', 'Awards'],
      dtype='object')

In [5]:
print(shoot.head())
shoot.columns

   Rk             Player  Age Team Pos   G  GS   MP    FG%  AvgFGDist  ...  \
0   1       De'Aaron Fox   27  SAC  PG  24  24  900  0.487       14.0  ...   
1   2      Keegan Murray   24  SAC  PF  24  24  885  0.422       16.1  ...   
2   3  Tyrese Haliburton   24  IND  PG  24  24  847  0.421       20.2  ...   
3   4      Mikal Bridges   28  NYK  SF  22  22  841  0.472       16.5  ...   
4   5      Jalen Johnson   23  ATL  SF  23  23  836  0.508       11.0  ...   

   FG%_3P  %FG_Astd_2P  %FG_Astd_3P  %FGA_Dunks  Num_Dunks  %3PA_Heaves  \
0   0.336        0.281        0.588       0.016          8        0.132   
1   0.291        0.610        1.000       0.076         20        0.381   
2   0.343        0.400        0.507       0.012          3        0.063   
3   0.329        0.667        0.939       0.031          9        0.443   
4   0.356        0.618        0.946       0.107         35        0.298   

   3P%_Heaves  Num_Heaves_Att  Num_Heaves_Made  Awards  
0       0.250          

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       'AvgFGDist', '%FGA_2P', '%FGA_0-3', '%FGA_3-10', '%FGA_10-16',
       '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3', 'FG%_3-10', 'FG%_10-16',
       'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_Dunks',
       'Num_Dunks', '%3PA_Heaves', '3P%_Heaves', 'Num_Heaves_Att',
       'Num_Heaves_Made', 'Awards'],
      dtype='object')

In [6]:
print(adv.head())
adv.columns

   Rk             Player  Age Team Pos   G  GS   MP   PER    TS%  ...  USG%  \
0   1       De'Aaron Fox   27  SAC  PG  24  24  900  20.2  0.583  ...  30.4   
1   2      Keegan Murray   24  SAC  PF  24  24  885  10.7  0.514  ...  15.4   
2   3  Tyrese Haliburton   24  IND  PG  24  24  847  17.9  0.563  ...  21.8   
3   4      Mikal Bridges   28  NYK  SF  22  22  841  12.6  0.553  ...  19.0   
4   5      Jalen Johnson   23  ATL  SF  23  23  836  17.5  0.585  ...  22.5   

   OWS  DWS   WS  WS/48  OBPM  DBPM  BPM  VORP  Awards  
0  1.3  0.8  2.2  0.115   2.5  -1.1  1.4   0.8     NaN  
1  0.4  0.7  1.1  0.062  -1.7  -1.5 -3.2  -0.3     NaN  
2  1.8  0.3  2.1  0.118   3.2  -1.6  1.6   0.8     NaN  
3  0.6  0.5  1.1  0.063  -0.4  -1.4 -1.8   0.0     NaN  
4  0.7  1.0  1.6  0.094   0.7   0.9  1.6   0.8     NaN  

[5 rows x 29 columns]


Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'Awards'],
      dtype='object')

In [7]:
condperg = perg[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 
                 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
                 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']]
condpbp = pbp[['Player', 'PG%', 'SG%', 'SF%', 'PF%', 'C%', 'OnCourt', 'On-Off', 
               'BadPassTO', 'LostBallTO', 'ShootFoulCmtd', 'OffFoulCmtd', 'ShootFoulDrawn', 
               'OffFoulDrawn', 'PtsGenbyAsts', 'And1s', 'FGA_Blkd']]
condshoot = shoot[['Player', 'AvgFGDist', '%FGA_2P', '%FGA_0-3', '%FGA_3-10', '%FGA_10-16',
                   '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3', 'FG%_3-10', 'FG%_10-16',
                   'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_Dunks',
                   'Num_Dunks', '%3PA_Heaves', '3P%_Heaves', 'Num_Heaves_Att', 'Num_Heaves_Made']]
condadv = adv[['Player', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 
               'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]

In [8]:
merged = pd.merge(condperg, condpbp, on='Player', how='inner').merge(condshoot, on='Player', how='inner').merge(condadv, on='Player', how='inner')
print(merged.head())
merged.columns

                    Player  Age Team Pos   G  GS    MP    FG   FGA    FG%  \
0    Giannis Antetokounmpo   30  MIL  PF  20  20  34.9  12.8  20.9  0.610   
1              LaMelo Ball   23  CHO  PG  18  18  34.1  10.7  24.9  0.430   
2             Nikola Jokić   29  DEN   C  17  17  37.7  11.4  20.4  0.559   
3  Shai Gilgeous-Alexander   26  OKC  PG  22  22  34.1  10.5  20.6  0.507   
4           Paolo Banchero   22  ORL  PF   5   5  36.4   9.6  19.4  0.495   

   ...  TOV%  USG%  OWS  DWS   WS  WS/48  OBPM  DBPM   BPM  VORP  
0  ...  11.5  36.7  2.7  0.9  3.6  0.246   6.9   2.2   9.1   1.9  
1  ...  14.1  39.8  0.9  0.4  1.4  0.108   6.0  -0.6   5.3   1.1  
2  ...  13.8  30.1  3.0  0.9  3.9  0.291  10.4   3.1  13.5   2.5  
3  ...   9.8  33.4  2.7  1.6  4.3  0.275   7.7   2.6  10.3   2.3  
4  ...   8.2  32.0  0.6  0.3  0.9  0.225   5.7   0.7   6.4   0.4  

[5 rows x 86 columns]


Index(['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PG%',
       'SG%', 'SF%', 'PF%', 'C%', 'OnCourt', 'On-Off', 'BadPassTO',
       'LostBallTO', 'ShootFoulCmtd', 'OffFoulCmtd', 'ShootFoulDrawn',
       'OffFoulDrawn', 'PtsGenbyAsts', 'And1s', 'FGA_Blkd', 'AvgFGDist',
       '%FGA_2P', '%FGA_0-3', '%FGA_3-10', '%FGA_10-16', '%FGA_16-3P',
       '%FGA_3P', 'FG%_2P', 'FG%_0-3', 'FG%_3-10', 'FG%_10-16', 'FG%_16-3P',
       'FG%_3P', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_Dunks', 'Num_Dunks',
       '%3PA_Heaves', '3P%_Heaves', 'Num_Heaves_Att', 'Num_Heaves_Made', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP'],
      dtype='object')

In [9]:
file_path = 'nba_player_stats_thru_12_6.csv'
merged.to_csv(file_path, index=False)

print(f"DataFrame exported to {file_path}")

DataFrame exported to nba_player_stats_thru_12_6.csv


In [10]:
midrange = merged.copy()
midrange['Position_Group'] = 'Wing'
midrange.loc[(midrange['C%'] > 50) | ((midrange['PF%'] > 1) & (midrange['C%'] > 10)), 'Position_Group'] = 'Big'
midrange.loc[(midrange['PG%'] > 1) & (midrange['SF%'] < 10), 'Position_Group'] = 'Guard'

## Process and Analyze
### Midrange Scorers

In [11]:
midrange['Middy Score'] = (midrange['%FGA_10-16'] * midrange['FG%_10-16']) + (midrange['%FGA_16-3P'] * midrange['FG%_16-3P'])

scaler = MinMaxScaler()

midrange['%FGA_10-3P'] = midrange['%FGA_10-16'] + midrange['%FGA_16-3P']
midrange['FG%_10-3P'] = ((midrange['FGA'] * midrange['%FGA_10-16'] * midrange['FG%_10-16']) + (midrange['FGA'] * midrange['%FGA_16-3P'] * midrange['FG%_16-3P'])) / (midrange['FGA'] * (midrange['%FGA_10-16'] + midrange['%FGA_16-3P']))
midrange['PTS_Mid'] = (midrange['FGA'] * midrange['%FGA_10-3P'] * midrange['FG%_10-3P'] * 2)
midrange['Total_PTS_Mid'] = midrange['PTS_Mid'] * midrange['G']
midrange['PTS_Mid_Scaled'] = scaler.fit_transform(midrange[['Total_PTS_Mid']])
midrange['Pts_Adj_Middy_Score'] = midrange['Middy Score'] * midrange['PTS_Mid_Scaled']

top_middy = midrange.sort_values(by='Pts_Adj_Middy_Score', ascending=False)
top_middy[['Player', 'Age', 'Team', 'Position_Group', 'G', 'MP', '%FGA_10-3P', 'FG%_10-3P', 'Total_PTS_Mid', 'Pts_Adj_Middy_Score']].head(10)

,Player,Age,Team,Position_Group,G,MP,%FGA_10-3P,FG%_10-3P,Total_PTS_Mid,Pts_Adj_Middy_Score
28,DeMar DeRozan,35,SAC,Wing,19,36.0,0.627,0.492215,195.849617,0.308619
26,Brandon Ingram,27,NOP,Wing,17,33.9,0.383,0.491914,124.270619,0.119546
14,Devin Booker,28,PHO,Wing,21,36.9,0.302,0.499795,116.010947,0.089408
3,Shai Gilgeous-Alexander,26,OKC,Guard,22,34.1,0.243,0.526975,116.069052,0.075891
11,Kevin Durant,36,PHO,Big,13,35.7,0.384,0.453354,77.852154,0.069202
8,De'Aaron Fox,27,SAC,Guard,24,37.5,0.266,0.441654,113.908608,0.068328
15,Jalen Brunson,28,NYK,Guard,22,33.8,0.252,0.489655,93.926752,0.059178
34,Jalen Williams,23,OKC,Wing,22,31.7,0.236,0.534775,91.626282,0.059045
0,Giannis Antetokounmpo,30,MIL,Big,20,34.9,0.239,0.480079,95.921804,0.056196
16,Cam Thomas,23,BRK,Wing,17,33.4,0.243,0.514016,74.319070,0.047398


In [12]:
file_path1 = 'nba_player_stats_plus_thru_12_6.csv'
top_middy.to_csv(file_path1, index=False)

print(f"DataFrame exported to {file_path1}")

DataFrame exported to nba_player_stats_plus_thru_12_6.csv


In [13]:
mid_big = top_middy[top_middy['Position_Group'] == 'Big']
print("Top Mid-Range Shooting Bigs")
mid_big[['Player', 'Age', 'Team', 'Position_Group', 'G', 'MP', '%FGA_10-3P', 'FG%_10-3P', 'Total_PTS_Mid', 'Pts_Adj_Middy_Score']].head(10)

Top Mid-Range Shooting Bigs


,Player,Age,Team,Position_Group,G,MP,%FGA_10-3P,FG%_10-3P,Total_PTS_Mid,Pts_Adj_Middy_Score
11,Kevin Durant,36,PHO,Big,13,35.7,0.384,0.453354,77.852154,0.069202
0,Giannis Antetokounmpo,30,MIL,Big,20,34.9,0.239,0.480079,95.921804,0.056196
99,Deandre Ayton,26,POR,Big,16,29.9,0.319,0.400078,48.191200,0.031404
7,Anthony Davis,31,LAL,Big,22,35.5,0.242,0.333273,65.650728,0.027035
53,Alperen Sengun,22,HOU,Big,23,30.8,0.175,0.451429,55.963600,0.022574
37,Nikola Vučević,34,CHI,Big,24,31.8,0.134,0.531507,49.570512,0.018027
25,Victor Wembanyama,21,SAS,Big,18,32.5,0.168,0.400006,44.030095,0.015108
87,Bam Adebayo,27,MIA,Big,20,34.8,0.218,0.333229,40.099488,0.014874
131,Jonas Valančiūnas,32,WAS,Big,20,19.6,0.238,0.385000,30.054640,0.014061
81,Jakob Poeltl,29,TOR,Big,22,32.7,0.122,0.579754,35.789380,0.012925


In [14]:
mid_wing = top_middy[top_middy['Position_Group'] == 'Wing']
print("Top Mid-Range Shooting Wings")
mid_wing[['Player', 'Age', 'Team', 'Position_Group', 'G', 'MP', '%FGA_10-3P', 'FG%_10-3P', 'Total_PTS_Mid', 'Pts_Adj_Middy_Score']].head(10)

Top Mid-Range Shooting Wings


,Player,Age,Team,Position_Group,G,MP,%FGA_10-3P,FG%_10-3P,Total_PTS_Mid,Pts_Adj_Middy_Score
28,DeMar DeRozan,35,SAC,Wing,19,36.0,0.627,0.492215,195.849617,0.308619
26,Brandon Ingram,27,NOP,Wing,17,33.9,0.383,0.491914,124.270619,0.119546
14,Devin Booker,28,PHO,Wing,21,36.9,0.302,0.499795,116.010947,0.089408
34,Jalen Williams,23,OKC,Wing,22,31.7,0.236,0.534775,91.626282,0.059045
16,Cam Thomas,23,BRK,Wing,17,33.4,0.243,0.514016,74.319070,0.047398
75,Mikal Bridges,28,NYK,Wing,22,38.2,0.228,0.520211,75.671904,0.045827
123,Bobby Portis,29,MIL,Wing,21,23.4,0.302,0.432195,64.139002,0.042745
45,Pascal Siakam,30,IND,Wing,24,34.4,0.250,0.432360,76.268304,0.042093
203,Kevin Porter Jr.,24,LAC,Wing,21,18.7,0.355,0.378549,50.233113,0.034468
128,Brandon Boston Jr.,23,NOP,Wing,20,27.9,0.297,0.426131,52.143132,0.033696


In [15]:
mid_guard = top_middy[top_middy['Position_Group'] == 'Guard']
print("Top Mid-Range Shooting Guards")
mid_guard[['Player', 'Age', 'Team', 'Position_Group', 'G', 'MP', '%FGA_10-3P', 'FG%_10-3P', 'Total_PTS_Mid', 'Pts_Adj_Middy_Score']].head(10)

Top Mid-Range Shooting Guards


,Player,Age,Team,Position_Group,G,MP,%FGA_10-3P,FG%_10-3P,Total_PTS_Mid,Pts_Adj_Middy_Score
3,Shai Gilgeous-Alexander,26,OKC,Guard,22,34.1,0.243,0.526975,116.069052,0.075891
8,De'Aaron Fox,27,SAC,Guard,24,37.5,0.266,0.441654,113.908608,0.068328
15,Jalen Brunson,28,NYK,Guard,22,33.8,0.252,0.489655,93.926752,0.059178
64,Jamal Murray,27,DEN,Guard,17,36.1,0.315,0.407724,65.937502,0.043240
164,Chris Paul,39,SAS,Guard,23,28.8,0.262,0.571969,55.836346,0.042724
44,Darius Garland,25,CLE,Guard,22,30.1,0.221,0.492462,71.830440,0.039916
159,T.J. McConnell,32,IND,Guard,23,19.8,0.289,0.482612,55.817895,0.039751
6,Luka Dončić,25,DAL,Guard,17,36.6,0.231,0.420818,74.034374,0.036747
10,Damian Lillard,34,MIL,Guard,19,36.1,0.194,0.522541,69.724349,0.036090
21,Kyrie Irving,32,DAL,Guard,21,34.9,0.217,0.438922,72.005976,0.035018


* To analyze mid-range scoring performance, I combined several metrics to create a composite score that effectively captures a player’s mid-range shooting proficiency and its contribution to their overall scoring. The process involved a combination of calculations, normalization, and scaling to ensure fairness and accuracy in comparing players.

* I started by defining the mid-range area as shots taken from 10 feet to the three-point line. Using this definition, I aggregated the percentage of field goal attempts taken from two ranges—10-16 feet (%FGA_10-16) and 16 feet to the three-point line (%FGA_16-3P)—into a new metric, %FGA_10-3P. This allowed me to evaluate a player's reliance on mid-range shooting as a percentage of their overall shot attempts. Next, I calculated the combined field goal percentage for the mid-range (FG%_10-3P) by weighting each range’s field goal percentage by its respective shot volume. This weighted approach provided a more accurate measure of mid-range efficiency.

* To quantify scoring output, I calculated the average points scored per game from mid-range shots (PTS_Mid) by multiplying mid-range field goal attempts, field goal percentage, and a factor of 2 (since each mid-range shot is worth two points). I then extended this metric across the total number of games played to determine the total points scored from mid-range (Total_PTS_Mid). This gave a cumulative view of each player’s mid-range contribution throughout the season.

* Since total mid-range points can vary significantly depending on playing time and shot volume, I normalized this metric using MinMaxScaler, creating a scaled version (PTS_Mid_Scaled) that fits within a 0 to 1 range. This ensured that comparisons between players were not biased by raw totals alone. Finally, I adjusted the existing “Middy Score” metric by multiplying it with the scaled mid-range points (PTS_Mid_Scaled), producing the final Pts_Adj_Middy_Score. This composite score reflects both efficiency and volume, providing a balanced measure of mid-range scoring performance.

* This process allowed me to distill multiple aspects of mid-range scoring—volume, efficiency, and game-to-game contributions—into a single, interpretable metric. By combining advanced calculations with normalization techniques, I ensured that the analysis was both comprehensive and equitable, making it easier to compare players across different playing styles and contexts.

### Defensive Anchors

In [16]:
defense = merged.copy()
condef = defense[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'DRB', 'STL', 'BLK', 'PF', 
                  'PG%', 'SG%', 'SF%', 'PF%', 'C%', 'ShootFoulCmtd', 'OffFoulDrawn', 
                  'DRB%', 'STL%', 'BLK%', 'DWS', 'DBPM']]
condefc = condef[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'DRB', 'STL', 'BLK', 'PF', 'ShootFoulCmtd', 'OffFoulDrawn', 'DRB%', 'STL%', 'BLK%', 'DWS', 'DBPM']]
condefc

,Player,Age,Team,Pos,G,GS,MP,DRB,STL,BLK,PF,ShootFoulCmtd,OffFoulDrawn,DRB%,STL%,BLK%,DWS,DBPM
0,Giannis Antetokounmpo,30,MIL,PF,20,20,34.9,9.5,0.5,1.3,2.5,19,2,28.6,0.7,3.3,0.9,2.2
1,LaMelo Ball,23,CHO,PG,18,18,34.1,4.4,1.1,0.2,4.1,22,6,14.9,1.6,0.7,0.4,-0.6
2,Nikola Jokić,29,DEN,C,17,17,37.7,9.5,1.8,0.8,1.9,10,2,27.0,2.2,2.0,0.9,3.1
3,Shai Gilgeous-Alexander,26,OKC,PG,22,22,34.1,4.5,1.8,1.0,2.0,26,1,13.8,2.5,3.1,1.6,2.6
4,Paolo Banchero,22,ORL,PF,5,5,36.4,6.4,0.6,0.8,2.6,9,0,21.1,0.8,2.1,0.3,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Enrique Freeman,24,IND,PF,14,1,7.6,0.6,0.1,0.1,1.1,10,1,8.7,0.9,1.8,0.0,-1.8
393,Armel Traoré,22,LAL,SF,7,0,7.7,1.9,0.6,0.3,0.9,0,0,27.5,3.6,3.4,0.1,-0.6
394,Jordan Walsh,20,BOS,PF,16,1,8.8,0.8,0.3,0.4,0.7,6,0,9.5,1.4,3.8,0.2,0.5
395,Ariel Hukporti,22,NYK,C,10,0,8.2,1.5,0.0,0.8,1.4,12,0,20.4,0.0,9.6,0.1,1.7


In [17]:
# Feature Engineering
columns = ['DRB', 'STL', 'BLK', 'PF', 'ShootFoulCmtd', 'OffFoulDrawn', 'DRB%', 'STL%', 'BLK%', 'DWS', 'DBPM']

scaler1 = MinMaxScaler()
condef1 = condef.copy()
condef1[columns] = scaler1.fit_transform(condef[columns])
condef1['DefensiveScore'] = condef1[columns].mean(axis=1)

condef1['Position_Group'] = 'Wing'
condef1.loc[(condef1['C%'] > 50) | ((condef1['PF%'] > 1) & (condef1['C%'] > 10)), 'Position_Group'] = 'Big'
condef1.loc[(condef1['PG%'] > 1) & (condef1['SF%'] < 10), 'Position_Group'] = 'Guard'
condef1

,Player,Age,Team,Pos,G,GS,MP,DRB,STL,BLK,...,C%,ShootFoulCmtd,OffFoulDrawn,DRB%,STL%,BLK%,DWS,DBPM,DefensiveScore,Position_Group
0,Giannis Antetokounmpo,30,MIL,PF,20,20,34.9,0.919192,0.161290,0.382353,...,36.0,0.463415,0.090909,0.754658,0.134615,0.272727,0.555556,0.694118,0.456234,Big
1,LaMelo Ball,23,CHO,PG,18,18,34.1,0.404040,0.354839,0.058824,...,0.0,0.536585,0.272727,0.329193,0.307692,0.057851,0.277778,0.364706,0.360385,Guard
2,Nikola Jokić,29,DEN,C,17,17,37.7,0.919192,0.580645,0.235294,...,100.0,0.243902,0.090909,0.704969,0.423077,0.165289,0.555556,0.800000,0.468612,Big
3,Shai Gilgeous-Alexander,26,OKC,PG,22,22,34.1,0.414141,0.580645,0.294118,...,0.0,0.634146,0.045455,0.295031,0.480769,0.256198,0.944444,0.741176,0.467969,Guard
4,Paolo Banchero,22,ORL,PF,5,5,36.4,0.606061,0.193548,0.235294,...,2.0,0.219512,0.000000,0.521739,0.153846,0.173554,0.222222,0.517647,0.314437,Wing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Enrique Freeman,24,IND,PF,14,1,7.6,0.020202,0.032258,0.029412,...,0.0,0.243902,0.045455,0.136646,0.173077,0.148760,0.055556,0.223529,0.121779,Wing
393,Armel Traoré,22,LAL,SF,7,0,7.7,0.151515,0.193548,0.088235,...,0.0,0.000000,0.000000,0.720497,0.692308,0.280992,0.111111,0.364706,0.252945,Wing
394,Jordan Walsh,20,BOS,PF,16,1,8.8,0.040404,0.096774,0.117647,...,0.0,0.146341,0.000000,0.161491,0.269231,0.314050,0.166667,0.494118,0.175902,Wing
395,Ariel Hukporti,22,NYK,C,10,0,8.2,0.111111,0.000000,0.235294,...,100.0,0.292683,0.000000,0.500000,0.000000,0.793388,0.111111,0.635294,0.271507,Big


In [18]:
top_def = condef1.sort_values(by='DefensiveScore', ascending=False)
top_def

,Player,Age,Team,Pos,G,GS,MP,DRB,STL,BLK,...,C%,ShootFoulCmtd,OffFoulDrawn,DRB%,STL%,BLK%,DWS,DBPM,DefensiveScore,Position_Group
25,Victor Wembanyama,21,SAS,C,18,18,32.5,0.797980,0.419355,1.000000,...,100.0,0.536585,0.227273,0.717391,0.365385,0.776860,0.666667,0.764706,0.616820,Big
53,Alperen Sengun,22,HOU,C,23,23,30.8,0.686869,0.387097,0.323529,...,100.0,0.902439,0.318182,0.624224,0.346154,0.272727,0.944444,0.800000,0.567881,Big
34,Jalen Williams,23,OKC,SG,22,22,31.7,0.484848,0.645161,0.264706,...,2.0,0.780488,0.136364,0.400621,0.596154,0.231405,1.000000,0.788235,0.542637,Wing
147,Tari Eason,23,HOU,PF,22,0,22.9,0.383838,0.677419,0.294118,...,5.0,0.536585,0.045455,0.459627,0.846154,0.338843,0.833333,0.917647,0.536102,Wing
81,Jakob Poeltl,29,TOR,C,22,22,32.7,0.727273,0.354839,0.382353,...,100.0,1.000000,0.090909,0.664596,0.307692,0.314050,0.555556,0.458824,0.525379,Big
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,Cam Whitmore,20,HOU,SF,8,0,9.9,0.050505,0.032258,0.029412,...,3.0,0.073171,0.000000,0.155280,0.115385,0.090909,0.111111,0.188235,0.097912,Wing
294,Antonio Reeves,24,NOP,SG,12,0,10.8,0.060606,0.096774,0.000000,...,0.0,0.121951,0.000000,0.192547,0.288462,0.000000,0.055556,0.094118,0.089721,Wing
383,Duop Reath,28,POR,C,13,0,6.4,0.010101,0.000000,0.058824,...,98.0,0.195122,0.000000,0.164596,0.000000,0.190083,0.055556,0.105882,0.084910,Big
355,Nick Smith Jr.,20,CHO,SG,11,0,9.0,0.030303,0.064516,0.000000,...,0.0,0.024390,0.045455,0.155280,0.192308,0.000000,0.055556,0.270588,0.078549,Wing


In [19]:
print(top_def.columns)

Index(['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'DRB', 'STL', 'BLK',
       'PF', 'PG%', 'SG%', 'SF%', 'PF%', 'C%', 'ShootFoulCmtd', 'OffFoulDrawn',
       'DRB%', 'STL%', 'BLK%', 'DWS', 'DBPM', 'DefensiveScore',
       'Position_Group'],
      dtype='object')


In [20]:
def_guard = top_def[top_def['Position_Group'] == 'Guard']
def_wing = top_def[top_def['Position_Group'] == 'Wing']
def_big = top_def[top_def['Position_Group'] == 'Big']

In [21]:
print("Top and Bottom 5 Defensive Guards")
def_guard[['Player', 'Age', 'Team', 'Position_Group', 'G', 'GS', 'MP', 'DefensiveScore']]

Top and Bottom 5 Defensive Guards


,Player,Age,Team,Position_Group,G,GS,MP,DefensiveScore
89,Jalen Suggs,23,ORL,Guard,24,24,30.0,0.501528
3,Shai Gilgeous-Alexander,26,OKC,Guard,22,22,34.1,0.467969
6,Luka Dončić,25,DAL,Guard,17,17,36.6,0.466094
255,Kris Dunn,30,LAC,Guard,24,13,22.6,0.448478
35,James Harden,35,LAC,Guard,24,24,33.9,0.438653
...,...,...,...,...,...,...,...,...
384,Jeff Dowtin,27,PHI,Guard,8,0,10.0,0.136076
314,Jared Butler,24,WAS,Guard,15,0,9.1,0.131258
336,Aaron Holiday,28,HOU,Guard,13,0,8.8,0.121881
211,Jaylen Nowell,25,NOP,Guard,8,0,21.0,0.120921


In [22]:
print("Top and Bottom 5 Defensive Wings")
def_wing[['Player', 'Age', 'Team', 'Position_Group', 'G', 'GS', 'MP', 'DefensiveScore']]

Top and Bottom 5 Defensive Wings


,Player,Age,Team,Position_Group,G,GS,MP,DefensiveScore
34,Jalen Williams,23,OKC,Wing,22,22,31.7,0.542637
147,Tari Eason,23,HOU,Wing,22,0,22.9,0.536102
107,Dyson Daniels,21,ATL,Wing,22,22,33.7,0.518507
18,Franz Wagner,23,ORL,Wing,25,25,33.2,0.491596
296,Alex Caruso,30,OKC,Wing,14,0,19.6,0.461781
...,...,...,...,...,...,...,...,...
391,Olivier-Maxence Prosper,22,DAL,Wing,13,0,5.7,0.114777
365,Doug McDermott,33,SAC,Wing,18,2,8.1,0.100167
276,Cam Whitmore,20,HOU,Wing,8,0,9.9,0.097912
294,Antonio Reeves,24,NOP,Wing,12,0,10.8,0.089721


In [23]:
print("Top and Bottom 5 Defensive Bigs")
def_big[['Player', 'Age', 'Team', 'Position_Group', 'G', 'GS', 'MP', 'DefensiveScore']]

Top and Bottom 5 Defensive Bigs


,Player,Age,Team,Position_Group,G,GS,MP,DefensiveScore
25,Victor Wembanyama,21,SAS,Big,18,18,32.5,0.616820
53,Alperen Sengun,22,HOU,Big,23,23,30.8,0.567881
81,Jakob Poeltl,29,TOR,Big,22,22,32.7,0.525379
199,Draymond Green,34,GSW,Big,20,19,28.4,0.524240
138,Isaiah Hartenstein,26,OKC,Big,7,6,29.7,0.517616
...,...,...,...,...,...,...,...,...
319,Isaac Jones,24,SAC,Big,14,0,9.8,0.175373
372,Dillon Jones,23,OKC,Big,20,0,8.8,0.165991
356,Tristan Thompson,33,CLE,Big,8,0,7.3,0.164202
330,Marvin Bagley III,25,WAS,Big,14,1,6.7,0.136232


* To calculate the Defensive Score, I began by selecting a set of key metrics that best represent a player's defensive impact. These included both basic per game statistics, such as defensive rebounds (DRB), steals (STL), blocks (BLK), personal fouls (PF), season totals for shooting fouls committed (ShootFoulCmtd), and offensive fouls drawn (OffFoulDrawn), as well as advanced metrics like defensive rebounding percentage (DRB%), steal percentage (STL%), block percentage (BLK%), defensive win shares (DWS), and defensive box plus-minus (DBPM). Together, these metrics provide a comprehensive view of a player's defensive contributions.

* To ensure that each metric contributed equally to the final score, I normalized their values to a common scale of 0 to 1 using the MinMaxScaler from the sklearn library. Normalization was essential because these metrics have different units and ranges, which could otherwise skew the results. By scaling the data, the lowest value in each metric became 0, and the highest value became 1, allowing for fair comparisons across metrics.

* Once the data was normalized, I calculated the Defensive Score by taking the mean of the normalized values for each player across all selected metrics. This approach gives equal weight to all metrics, ensuring that no single statistic dominates the score. The result is a single numeric value on a 0 to 1 scale that represents a player’s overall defensive performance, with higher scores indicating stronger defensive contributions.

* This process was straightforward yet effective. It allowed me to distill complex and diverse defensive metrics into an interpretable and balanced score. By avoiding arbitrary weights and ensuring fairness through normalization, the Defensive Score provides a transparent and meaningful way to evaluate and compare players' defensive impacts.